# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd

In [ ]:
data_0 = pd.read_csv('geo_data_0.csv')
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_0['product'].max(), data_0['product'].min()

(185.3643474222929, 0.0)

In [ ]:
data_1 = pd.read_csv('geo_data_1.csv')
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_1['product'].max(), data_1['product'].min()

(137.94540774090564, 0.0)

In [ ]:
data_2 = pd.read_csv('geo_data_2.csv')
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_2['product'].max(), data_2['product'].min()

(190.0298383433513, 0.0)

С данными все в порядке, пропусков нет, все значения в нужном типе данных, так же неправдаподобных значений не наблюдаю

## Обучение и проверка модели

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
def train_f(data, region):
    features = data.drop(['product', 'id'], axis=1)
    target = data['product']
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=123)
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, predictions)
    avg_predicted = predictions.mean()
    print(f"Регион: {region}")
    print(f"Средний предсказанный запас: {avg_predicted:.2f} тыс. баррелей")
    print(f"RMSE: {rmse:.2f}")
    return pd.Series(predictions, index=y_valid.index), y_valid.reset_index(drop=True)

In [ ]:
pred_0, y_valid_0 = train_f(data_0, "Регион 0")
pred_1, y_valid_1 = train_f(data_1, "Регион 1")
pred_2, y_valid_2 = train_f(data_2, "Регион 2")

Регион: Регион 0
Средний предсказанный запас: 92.55 тыс. баррелей
RMSE: 1417.36
Регион: Регион 1
Средний предсказанный запас: 69.28 тыс. баррелей
RMSE: 0.80
Регион: Регион 2
Средний предсказанный запас: 95.10 тыс. баррелей
RMSE: 1610.26


RMSE в регионе 1 наиментший, значит модель там более точная

## Подготовка к расчёту прибыли

In [ ]:
BUDGET = 10000000000
INCOME_PER_BARREL = 450000
N_POINTS = 500
N_BEST = 200

In [ ]:
min_oil_per_well = BUDGET / (N_BEST * INCOME_PER_BARREL)
print(f"Минимальный объём на скважину для безубыточности: {min_oil_per_well:.2f} тыс. баррелей")

Минимальный объём на скважину для безубыточности: 111.11 тыс. баррелей


In [ ]:
print(f"Средний запас в Регионе 0: {data_0['product'].mean():.2f}")
print(f"Средний запас в Регионе 1: {data_1['product'].mean():.2f}")
print(f"Средний запас в Регионе 2: {data_2['product'].mean():.2f}")

Средний запас в Регионе 0: 92.50
Средний запас в Регионе 1: 68.83
Средний запас в Регионе 2: 95.00


Средние запасы во всех регионах ниже безубыточного уровня.

## Расчёт прибыли и рисков

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def calculate_profit(target, predictions, n_best=200):
    best_indices = np.argsort(predictions)[-n_best:]
    total_product = target.iloc[best_indices].sum()
    revenue = total_product * INCOME_PER_BARREL
    profit = revenue - BUDGET
    return profit

In [ ]:
profit_0 = calculate_profit(pred_0, y_valid_0)
profit_1 = calculate_profit(pred_1, y_valid_1)
profit_2 = calculate_profit(pred_2, y_valid_2)
print(profit_0, profit_1, profit_2)

369991429.334568 2370614615.685934 -449042601.72203827


In [ ]:
def bootstrap_analysis(predictions, target, region_name, n_samples=1000):
    predictions = np.array(predictions)
    target = np.array(target)

    profits = []
    rng = np.random.RandomState(12345)

    for _ in range(n_samples):
        idx = rng.randint(0, len(target), size=len(target))
        pred_subset = predictions[idx]
        target_subset = target[idx]
        temp_df = pd.DataFrame({
            'predictions': pred_subset,
            'target': target_subset
        })
        profit = calculate_profit(temp_df['target'], temp_df['predictions'], N_BEST)
        profits.append(profit)

    profits = pd.Series(profits)
    mean_profit = profits.mean()
    ci_low, ci_high = profits.quantile([0.025, 0.975])
    risk = (profits < 0).mean() * 100

    print(f"Регион: {region_name}")
    print(f"Средняя прибыль: {mean_profit:.2f} руб.")
    print(f"95% Доверительный интервал: [{ci_low:.2f}, {ci_high:.2f}] руб.")
    print(f"Риск убытков: {risk:.1f}%")
    print('- - - - - - - - - - - - - - - -')
    return profits

In [ ]:
profits_0 = bootstrap_analysis(pred_0, y_valid_0, "Регион 0")
profits_1 = bootstrap_analysis(pred_1, y_valid_1, "Регион 1")
profits_2 = bootstrap_analysis(pred_2, y_valid_2, "Регион 2")

Регион: Регион 0
Средняя прибыль: 3539620255.39 руб.
95% Доверительный интервал: [3236136733.18, 3822878109.14] руб.
Риск убытков: 0.0%
- - - - - - - - - - - - - - - -
Регион: Регион 1
Средняя прибыль: 2415086696.68 руб.
95% Доверительный интервал: [2415086696.68, 2415086696.68] руб.
Риск убытков: 0.0%
- - - - - - - - - - - - - - - -
Регион: Регион 2
Средняя прибыль: 2380717876.32 руб.
95% Доверительный интервал: [2011888626.66, 2752338647.70] руб.
Риск убытков: 0.0%
- - - - - - - - - - - - - - - -


Учитывая как среднюю ожидаемую прибыль, так и риск убытков, а также точность модели, наиболее взвешенным решением будет рассмотреть Регион 0 как основной приоритет из-за самой высокой потенциальной прибыли при нулевом риске убытков.<br>

Регион 1 также является очень привлекательным вариантом благодаря высокой точности модели и значительной средней прибыли с нулевым риском убытков. Низкий RMSE и стабильная оценка прибыли в Bootstrap, делает его надежным вариантом.<br>

Регион 2, несмотря на положительную среднюю прибыль в Bootstrap, выглядит наименее предпочтительным из-за низкой точности модели и отрицательного результата единовременного расчета.<br>

Рекомендуется рассмотреть для разработки в первую очередь Регион 0 из-за наибольшей потенциальной прибыли. Регион 1 также является очень привлекательным вариантом с высокой точностью прогноза и значительной прибылью, что может сделать его более стабильным выбором с точки зрения оценки запасов. Регион 2 следует рассматривать с осторожностью из-за низкой точности модели.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован